# Colección de datos para seguimiento carretera - **Anthony**
---
La idea es que Anthony pueda circular por una carretera respetando las vías de tránsito.

# Impotación de librerías

---

Se utilizará OpenCV para visualizar y almacenar las imágenes con los labels, las librerías UUID y date time para nombrar las imágenes.


In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import glob
import datetime
import numpy as np
import cv2
import time

# Camara en vivo

---

La red neuronal toma imágenes de 224x224, por lo tanto se ajustará el tamaño de la cámara a este tamaño para no tener problemas.

Se muestra la cámara en vivo de Anthony.

In [2]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([target_widget]), x_slider, y_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0, step=0.001)

Se crea la carpeta que contendrá el dataset (en caso de no existir). Además, se crea un contador para llevar la cuenta de las imágenes tomadas, así como un botón que realice la toma de las imágenes.

In [3]:
DATASET_DIR = 'dataset'

try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

button_layout = widgets.Layout(width='100px', height='64px')    

count_widget = widgets.IntText(description='Contador', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))
take_photo = widgets.Button(description='Tomar', button_style='warning', layout=button_layout)

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot():
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

take_photo.on_click(lambda x: save_snapshot())

display(widgets.HBox([target_widget]), x_slider, y_slider)
display(widgets.VBox([count_widget]))
display(widgets.HBox([take_photo]))

Directories not created becasue they already exist


FloatSlider(value=-0.183, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.619, description='y', max=1.0, min=-1.0, step=0.001)

In [5]:
camera.restart()

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}